A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
%test_bs1flgm2flgm2f_20180803_uszc_LIB.m


clc
clear
% valueDate = '2018-01-09';
valueDate = '2017-02-17';

%simulation settin g
maxIdx = 10;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
maxIdx = 0;
zcStep = 0;
shiftAmount = 0;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

%% USDCMSRA KRWCMSSpreadRA Pricing start
%% CMS CMSRangeAccrual valuation start
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2017-02-17');
rangeAccrualParams.maturity = 15;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0362;
couponFreq = 1;
tenorL = 10;
tenorS = 2;
lowerS = 0.0;
upperS = 1;

tenor = 10;
lower = 0.0;
upper = 0.06;

rangeDays = zeros(15,1);


%resetDate startDate endDate paymentDate
couponScheduleInt = [20170217    20170217    20180217    20180219
20180214    20180217    20190217    20190218
20190215    20190217    20200217    20200217
20200217    20200217    20210217    20210217
20210217    20210217    20220217    20220217
20220217    20220217    20230217    20230217
20230217    20230217    20240217    20240219
20240216    20240217    20250217    20250217
20250217    20250217    20260217    20260219
20260213    20260217    20270217    20270217
20270217    20270217    20280217    20280217
20280217    20280217    20290217    20290219
20290216    20290217    20300217    20300218
20300215    20300217    20310217    20310217
20310217    20310217    20320217    20320217
];

%% LGM1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
%% end

callYN = ones(15,1);
callYN(1) = 0;
callYN(2) = 0;
callYN(3) = 0;
%resetDate startDate endDate paymentDate
callScheduleInt = [20170217    20170217    20180217    20180219
20180214    20180217    20190217    20190218
20190215    20190217    20200217    20200217
20200210    20200217    20210217    20200217
20210208    20210217    20220217    20210217
20220210    20220217    20230217    20220217
20230210    20230217    20240217    20230217
20240208    20240217    20250217    20240219
20250210    20250217    20260217    20250217
20260209    20260217    20270217    20260219
20270210    20270217    20280217    20270217
20280210    20280217    20290217    20280217
20290207    20290217    20300217    20290219
20300211    20300217    20310217    20300218
20310210    20310217    20320217    20310219
];


manualScheduleYN = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
        && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
    elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
        rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
    end
end

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lowerS','upperS','tenor','lower','upper',...
                                'rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lowerS, ...
                        upperS,tenor,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;

rangeAccrual = CMSCMSSpreadDualRangeAccrual(rangeAccrualParams);

%% Simulation Start
% for idx2=1:2*maxIdx+1
% for idx2=2*maxIdx+1:2*maxIdx+1
for idx2=maxIdx+1:maxIdx+1
    %% KRW Model building

    %% import and construct a zeroCurve
    rawData = [0.0027397 	0.999959728
    0.2500000 	0.995871845
    0.5000000 	0.99154544
    0.7500000 	0.986866679
    1.000000 	0.981892589
    2.000000 	0.960911374
    3.000000 	0.93966943
    4.000000 	0.918499662
    5.000000 	0.897462069
    7.000000 	0.856996516
    10.000000 	0.797414396
    12.000000 	0.758158208
    15.000000 	0.706176858
    20.000000 	0.628858172
    25.000000 	0.560374492
    30.000000 	0.499419054
        ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999959728 
    0.25	0.996247006 
    0.5	0.992160837 
    0.75	0.987265980 
    1	0.982295684 
    2	0.959567666 
    3	0.938442690 
    5	0.889131313 
    10	0.775743705 
    20	0.603326670 
    30	0.469009914 
        ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.180918	0.187412	0.191522	0.193809	0.197145	0.197977	0.202251	0.204123	0.204123
    2	0.201024	0.201271	0.197475	0.196981	0.195931	0.194313	0.195361	0.199344	0.199344
    3	0.206411	0.20229	0.197703	0.195155	0.193529	0.191295	0.19266	0.196655	0.196655
    4	0.204366	0.201238	0.195861	0.193106	0.191647	0.188532	0.191248	0.195619	0.195619
    5	0.208548	0.202396	0.19528	0.190753	0.189714	0.184224	0.18525	0.189847	0.189847
    7	0.202382	0.195093	0.189328	0.18233	0.177407	0.176157	0.177346	0.191365	0.191365
    10	0.189532	0.183121	0.178702	0.177781	0.174801	0.175461	0.177286	0.193966	0.193966
    15	0.20237	0.207678	0.1942	0.190125	0.188303	0.187428	0.18773	0.198563	0.198563
        ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);

     KRWAllMktData('swaptionVol') = swaptionVolMktData;
    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a LGM2FModel with initial modelParams
    tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
    dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
    dH2.tenor = dH1.tenor;

    %swaption only initial parameters
    % dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

    %Samsung
    % dHTest = [0.058507682	0.500210296
    % 0.06145709	0.515096302
    % 0.057841075	0.632558982
    % 0.069442883	0.632845436
    % 0.081752868	0.607322125
    % 0.087574376	0.586095839
    % 0.103284643	0.52688632
    % 0.113936628	0.552598585
    % 0.131046999	0.498409755
    % 0.153553385	0.437472215
    % 0.173381122	0.423805445
    % 0.180703372	0.380344606
    % 0.207484315	0.305927652
    % 0.218329282	0.268192317
    % 0.224885432	0.227191192
    % 0.221244544	0.231204789
    % 0.262502203	0.212209604
    % 0.264579505	0.135257044
    % 0.318821555	0.118179799
    % 0.271515922	0.107957745
    % 0.358024755	0.088834433
    % 0.33143161	0.069384916
    % 0.457059078	0.033745097
    % 0.374067262	0.006573072
    % ];

    %Daewoo
    % dHTest = [0.037286763	0.525314433
    % 0.037286763	0.525314433
    % 0.054571525	0.586928199
    % 0.069289617	0.618362181
    % 0.099111958	0.605543022
    % 0.1123501	0.589459841
    % 0.139006516	0.523225356
    % 0.175987354	0.505500158
    % 0.184616726	0.463242812
    % 0.204164537	0.412151488
    % 0.263444475	0.364947643
    % 0.251615512	0.358337407
    % 0.259708458	0.315375986
    % 0.269854863	0.270026782
    % 0.337623406	0.253701114
    % 0.258270519	0.179914465
    % 0.321985902	0.191343978
    % 0.347503907	0.154371663
    % 0.36490148	0.127052474
    % 0.36706888	0.095165077
    % 0.480469769	0.05084228
    % 0.423787026	0.033262976
    % 0.531944266	0.006463726
    % 0.333342463	0.000100001
    % ];

    % %Sungwoo
    % dHTest = [0.060654929	0.585843811
    % 0.065181321	0.621357434
    % 0.059213061	0.705433024
    % 0.070356009	0.697274699
    % 0.084854535	0.641491331
    % 0.078986949	0.60204605
    % 0.103979263	0.504724505
    % 0.128905871	0.522635178
    % 0.142868097	0.436948669
    % 0.167927649	0.341253642
    % 0.171414266	0.384418469
    % 0.173137707	0.332946905
    % 0.205465936	0.252922264
    % 0.190811162	0.233849244
    % 0.212134317	0.235149902
    % 0.208305697	0.261176412
    % 0.267225845	0.213982892
    % 0.244169602	0.131637885
    % 0.293167102	0.110338057
    % 0.302235653	0.107810085
    % 0.341199539	0.095697121
    % 0.353976	0.072468098
    % 0.421517994	0.033547027
    % 0.357938773	0.006568033
    % ];

    %JaeYeol
    dHTest = [0.060438773	0.585467789
    0.064928789	0.621766986
    0.059045192	0.705355899
    0.070178721	0.6972879
    0.084746775	0.641430112
    0.079080439	0.601700141
    0.1039792	0.504405369
    0.129262749	0.522031261
    0.14309303	0.436403222
    0.167840108	0.341211636
    0.171134327	0.384814515
    0.17295042	0.333229804
    0.205446544	0.253016302
    0.190785578	0.233852044
    0.212042588	0.235092969
    0.208563343	0.261106368
    0.26724335	0.213909029
    0.24410813	0.131731127
    0.292886806	0.110481688
    0.302159283	0.107800708
    0.341084552	0.095566454
    0.353944447	0.072422244
    0.421105867	0.033562964
    0.357823112	0.006568462
    ];

    %new 201809
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha1.tenor =  expiry;
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(expiry),1);

    %swaption only initial parameters

    %Samsung
    % AlphaTest = [0.0289581	0.006633509
    % 0.021585088	0.007003076
    % 0.019812596	0.007056061
    % 0.015014178	0.007801684
    % 0.008683651	0.008360616
    % 0.011075137	0.008248663
    % 0.008794173	0.011226664
    % 0.000329846	0.023982837
    % ];

    %Daewoo
    % AlphaTest = [0.029142908	0.007983918
    % 0.016018045	0.007219568
    % 0.00912537	0.007233927
    % 0.008101129	0.007774553
    % 0.012850209	0.008916743
    % 0.011065789	0.009477842
    % 0.008930619	0.011214209
    % 0.0001	0.032096513
    % ];

    % %sungwoo
    % AlphaTest = [0.032526561	0.005901286
    % 0.021888486	0.006159185
    % 0.015500653	0.006183779
    % 0.011483817	0.007702683
    % 0.008112119	0.008910715
    % 0.010323471	0.008310996
    % 0.006966815	0.012340985
    % 0.000302754	0.020086129
    % ];

    %jaeweol
    AlphaTest = [0.032529518	0.005904921
    0.021882297	0.0061619
    0.015498414	0.006184972
    0.011471555	0.007705108
    0.008105853	0.008915352
    0.010349202	0.008306695
    0.006993193	0.012332999
    0.000309834	0.020072884
    ];

    %201809
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];

    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});

    KRWLGM2F = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));


    tic
    out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc


    dH1Calib = KRWLGM2F.dH1;
    dH2Calib = KRWLGM2F.dH2;
    Alpha1Calib = KRWLGM2F.Alpha1;
    Alpha2Calib = KRWLGM2F.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
    %% LGM model building end

    %% KRW model building end

    %% USD model building start
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999960071
    0.083333333	0.998705285
    0.166666667	0.997278569
    0.25	0.99570906
    0.5	0.990659472
    0.75	0.985691562
    1	0.980706602
    2	0.958190889
    3	0.935244778
    4	0.91270743
    5	0.890529514
    7	0.846476648
    10	0.781398058
    12	0.739443263
    15	0.680504997
    20	0.593081133
    25	0.519315395
    30	0.456759825
        ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

     %simulation testCase
    %for idx=1:size(rawData,1)
    %    rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    %end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','USDZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    USDAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999960071
    0.083333333	0.998705285
    0.166666667	0.997278569
    0.25	0.99570906
    0.5	0.990659472
    0.75	0.985691562
    1	0.980706602
    2	0.958190889
    3	0.935244778
    4	0.91270743
    5	0.890529514
    7	0.846476648
    10	0.781398058
    12	0.739443263
    15	0.680504997
    20	0.593081133
    25	0.519315395
    30	0.456759825
        ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'USD','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);
    USDAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurface
    rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
    1	0.18785	0.21535	0.228	0.23405	0.23705	0.23735	0.2341	0.23	0.2259
    2	0.22675	0.24245	0.25075	0.25335	0.25235	0.2504	0.2464	0.23945	0.2325
    3	0.25885	0.26305	0.2647	0.263	0.26135	0.2569	0.2502	0.243325	0.23645
    4	0.2736	0.27245	0.2692	0.266	0.263	0.25715	0.2503	0.243125	0.23595
    5	0.27605	0.272	0.26905	0.26565	0.2627	0.2561	0.24885	0.24125	0.23365
    7	0.2656	0.26325	0.25995	0.25655	0.25355	0.24835	0.24125	0.233575	0.2259
    10	0.24575	0.2439	0.24245	0.23995	0.2385	0.23425	0.22875	0.2217	0.21465
    15	0.2192	0.21675	0.2156	0.21345	0.21175	0.20865	0.20585	0.2007	0.19555
        ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'USD','USDSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    USDAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a BlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    %% create a USD LGM2FModel with initial modelParams
    
    %swaption only initial parameters
    dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
    dH2.tenor = dH1.tenor;

    %JaeYeol
    dHTest = [0.060438773	0.585467789
    0.064928789	0.621766986
    0.059045192	0.705355899
    0.070178721	0.6972879
    0.084746775	0.641430112
    0.079080439	0.601700141
    0.1039792	0.504405369
    0.129262749	0.522031261
    0.14309303	0.436403222
    0.167840108	0.341211636
    0.171134327	0.384814515
    0.17295042	0.333229804
    0.205446544	0.253016302
    0.190785578	0.233852044
    0.212042588	0.235092969
    0.208563343	0.261106368
    0.26724335	0.213909029
    0.24410813	0.131731127
    0.292886806	0.110481688
    0.302159283	0.107800708
    0.341084552	0.095566454
    0.353944447	0.072422244
    0.421105867	0.033562964
    0.357823112	0.006568462
    ];

%new 201809
    dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
    ];

    dH1.quote = dHTest(:,1);
    dH2.quote = dHTest(:,2);


    dH1Orig = dH1;
    dH2Orig = dH2;

    Alpha1.tenor =  [1;2;3;4;5;7;10;15];
    Alpha2.tenor =  Alpha1.tenor;
    correl.tenor = Alpha1.tenor;
    correl.quote = zeros(length(Alpha1.tenor),1);

    %swaption only initial parameters
    %jaeweol
    AlphaTest = [0.032529518	0.005904921
    0.021882297	0.0061619
    0.015498414	0.006184972
    0.011471555	0.007705108
    0.008105853	0.008915352
    0.010349202	0.008306695
    0.006993193	0.012332999
    0.000309834	0.020072884
    ];

%201809
    AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
    ];


    Alpha1.quote = AlphaTest(:,1);
    Alpha2.quote = AlphaTest(:,2);

    smoothdH = 1.0;
    smoothAlpha = 1.0;
    smoothTermCorr =0.1;
    targetTermCorr = 0.92;

    freq = 4;

    modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                            {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});

    USDLGM2F = LGM2F(IRModel(Model(USDAllMktData,modelParams)));


    tic
    out = USDLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
    % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
    toc


    dH1Calib = USDLGM2F.dH1;
    dH2Calib = USDLGM2F.dH2;
    Alpha1Calib = USDLGM2F.Alpha1;
    Alpha2Calib = USDLGM2F.Alpha2;

    modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                            {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

    USDLGM2FRisky = LGM2FRisky(IRModelRisky(Model(USDAllMktData,modelParamsCalib)));
%     
%     dH1.tenor = [1;2;3;4;5;7;10;15];
% 
%     dH1.quote = ones(1,length(dH1.tenor))*1.0;
%     dH1Prev = dH1;
%     Alpha1.tenor =  [1;2;3;4;5;7;10;15];
%     Alpha1.quote = ones(1,length(dH1.tenor))*0.01;
% 
%     joint_swap_tenor = 1;
% 
%     smoothdH = 1.0;
%     smoothAlpha = 0.00;
% 
%     targetSize = 15;
% 
%     freq = 4;
%     tic
%     modelParams= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
%                             {dH1,Alpha1,matu,joint_swap_tenor, 'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});
%     USDLGM1F = LGM1F(IRModel(Model(USDAllMktData,modelParams)));
% 
%     USDLGM = LGM1F(IRModel(Model(USDAllMktData,modelParams)));
%     % out = USDLGM.CalibrateToATMSwaptionTenorDiagSwaption(USDBlack,'global');
%     out = USDLGM1F.CalibrateToATMDiagSwaption(KRWBlack,'global');
%     aaa =1.0;
% 
%     dH1Calib = USDLGM.dH1;
%     Alpha1Calib = USDLGM.Alpha1;
% 
%     modelParamsCalib= containers.Map({'dH1','Alpha1','matu','joint_swap_tenor','modelName','dH1Prev','smoothdH','smoothAlpha','targetSize','freq'}, ...
%                         {dH1Calib,Alpha1Calib,matu,joint_swap_tenor,'LGM1F',dH1Prev,smoothdH,smoothAlpha,targetSize,freq});
% 
%     USDLGMRisky = LGM1FRisky(IRModelRisky(Model(USDAllMktData,modelParamsCalib)));

    %% USD model building end

    %% USDKRW fx model buidling start
       rawData = [0.083333333	0.0907
    0.25	0.0995
    0.5	0.0997
    1	0.1023
    2	0.1073
    3	0.1092
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','USDKRWATMTermVol'});
    impliedTermVolMktData = MktData(mktData);
    %     impliedTermVole = FXVolSurface(impliedTermVolMktData);
    %     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
    %     AllMktData('impliedVolSurface') = impliedTermVolMktData;

    %% create  an FXBlackModel to calculate market PlainVanilla Options
    %     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});

    %     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 

    %% USDKRW fx model building end

    %% MultiAssetModel building start
    multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                            {KRWAllMktData,USDAllMktData,USDKRWAllMktData});

    %KRW,USD & USDKRW
    correlationMatrix = ones(3,3);
%     correlationMatrix(1,2) = 0.3;
    correlationMatrix(1,2) = 0.108803681;
    
    correlationMatrix(1,3) = -0.17;
    correlationMatrix(2,3) = 0.1;
    correlationMatrix(2,1) = correlationMatrix(1,2);
    correlationMatrix(3,1) = correlationMatrix(1,3);
    correlationMatrix(3,2) = correlationMatrix(2,3);


    multiAssetMktData('correlationMatrix') = correlationMatrix;

    modelNameMap = containers.Map({'KRW','USD'},{KRWLGM2FRisky,USDLGM2F});
    modelParams= containers.Map({'modelNameMap'},{modelNameMap});
    % correlationNames = {'KRW','USD'};
    modelParams('stochasticModelNames') = {'KRW','USD'};
    modelParams('correlationMatrix') = correlationMatrix;

    modelParams('refModelName') = 'KRW';

    % modelParams('useQuasiRandomYN') = 'true';
    modelParams('useQuasiRandomYN') = 'false';
    
%     modelParams('useBrigoCorrYN') = 'true';
    modelParams('useBrigoCorr') = 'original';
%     modelParams('useBrigoCorr') = 'adjust';
%     modelParams('useBrigoCorr') = 'simple_adjust';
%     modelParams('useBrigoCorr') = 'simple';

    % KRWUSD Historical CrossCorrelation 
%     crossCorrelation_expiry = [1;2;3;5;10];
% KRW & USD
    crossCorrelation =  [0	1	2	3	5	10
1	0.102325534	0.096352535	0.085261116	0.068171708	0.060847819
2	0.106932815	0.10278167	0.095176815	0.086514389	0.092527673
3	0.10538545	0.107634217	0.10510726	0.098599496	0.112249122
5	0.125973831	0.126932702	0.124030498	0.12181951	0.140572686
10	0.12868268	0.127415811	0.124572602	0.128098107	0.146125966
];

% % Test KRW & KTB
%     crossCorrelation =  [0	1	2	3	5	10
% 1	0.733938323	0.797813102	0.804426421	0.757862429	0.639051049
% 2	0.722212969	0.885540894	0.911107206	0.883573272	0.781134839
% 3	0.691264397	0.884242638	0.929611332	0.924497839	0.848967713
% 5	0.599723785	0.817913987	0.887767389	0.931979088	0.924645192
% 10	0.490548154	0.725348245	0.823154971	0.895677869	0.950807001
% ];
% 
% % Test Rebonato Corr 0.4/0.15
%     crossCorrelation =  [0	1	2	3	5	10
% 1	1	0.9164	0.8445	0.7293	0.5555
% 2	0.9164	1	0.9164	0.7826	0.5807
% 3	0.8445	0.9164	1	0.8445	0.61
% 5	0.7293	0.7826	0.8445	1	0.6834
% 10	0.5555	0.5807	0.61	0.6834	1
% ];
    modelParams('crossCorrelation') = crossCorrelation;
    modelParams('useBrigoCorr') = 'calibrateToHist';
    modelParams('useBrigoCorr') = 'simple';
    
    KRWLGM2FRiskyUSDLGM2F = MultiAssetModelLGM2FLGM2F(multiAssetMktData,modelParams);
    
    

    %% MultiAssetModel building end

    %% NumMethod Init Start

    numOfRandom = 20000;

    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                            {'AMC',7,numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);

    %% NumMethod Init End
    pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRA(rangeAccrual,KRWLGM2FRiskyUSDLGM2F,numMethodAMC,false);

%         pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
    pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSA');

    pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
    closedPrice(idx2) = pricingOutClosed.pricerInfo.nonCall.npv;
    
    for idx3=1:length(pricingOutMC1.nonCall.cashflow_npv_e)
        nonCallPrice_cashflowNPV(idx2,idx3) = pricingOutMC1.nonCall.cashflow_npv_e(idx3);
        closedPrice_cashflowNPV(idx2,idx3) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(idx3);
    end
    
    newexpiry = [1;2;3;4;5;7;10;15];


    testCK.impCorrel = zeros(size(newexpiry,1),size(newexpiry,1));
    for i=1:size(newexpiry,1)
        for j=1:size(newexpiry,1)
            testCK.impCorrel(i,j)= KRWLGM2FRiskyUSDLGM2F.ImpCorrel(5,newexpiry(i),newexpiry(j));
        end
    end

end

aaaa =1.0;


In [ ]:
%test_bond_forward_maturityMatchingType_25_3.m


clc
clear
valueDate = '2023-04-24';
%% import and construct a zeroCurve
% SOFR zero
tNodes = [0.00547945
0.25479452
0.50684932
0.76438356
1.00547945
1.51232877
2.01095890
3.00821918
4.00547945
5.00821918
7.00821918
10.01095890
12.01095890
15.01369863
20.01917808
25.02465753
30.03013699
];

zeroRates = [0.035798244	0.035898235	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244	0.035798244
0.036033292	0.036034367	0.036131321	0.036033292	0.036033292	0.036033292	0.036033292	0.036033292	0.036033292	0.036033292	0.036033292	0.036033292
0.036121196	0.036121736	0.036120745	0.036220206	0.036121196	0.036121196	0.036121196	0.036121196	0.036121196	0.036121196	0.036121196	0.036121196
0.035784209	0.035784568	0.035783913	0.035783614	0.035883834	0.035784209	0.035784209	0.035784209	0.035784209	0.035784209	0.035784209	0.035784209
0.035285842	0.035286114	0.03528562	0.035285396	0.035285151	0.035386027	0.035285842	0.035285842	0.035285842	0.035285842	0.035285842	0.035285842
0.034179826	0.034180007	0.034179683	0.034179539	0.034179381	0.03417891	0.034280509	0.034179826	0.034179826	0.034179826	0.034179826	0.034179826
0.032992707	0.032992843	0.032992603	0.032992499	0.032992385	0.032992043	0.032991438	0.033094171	0.032992707	0.032992707	0.032992707	0.032992707
0.031523215	0.031523306	0.031523149	0.031523083	0.03152301	0.031522792	0.031522408	0.031521465	0.031625474	0.031523215	0.031523215	0.031523215
0.030893319	0.030893387	0.03089327	0.030893222	0.030893168	0.030893008	0.030892726	0.030892033	0.030890915	0.03099709	0.030893319	0.030893319
0.030489083	0.030489137	0.030489045	0.030489006	0.030488964	0.030488838	0.030488616	0.03048807	0.030487189	0.030486512	0.030594389	0.030489083
0.030462222	0.030462261	0.030462195	0.030462168	0.030462138	0.030462048	0.03046189	0.030461501	0.030460873	0.030460391	0.030458466	0.03056954
0.030705923	0.03070595	0.030705903	0.030705884	0.030705863	0.0307058	0.030705688	0.030705414	0.030704972	0.030704632	0.030703276	0.030699643
0.031001722	0.031001745	0.031001706	0.03100169	0.031001672	0.031001619	0.031001525	0.031001294	0.031000921	0.031000635	0.030999492	0.03099643
0.030876386	0.030876404	0.030876373	0.03087636	0.030876346	0.030876303	0.030876228	0.030876045	0.030875748	0.03087552	0.030874612	0.030872176
0.02992629	0.029926304	0.029926281	0.029926272	0.029926261	0.029926231	0.029926177	0.029926046	0.029925833	0.029925669	0.029925017	0.029923269
0.027827579	0.02782759	0.027827573	0.027827566	0.027827559	0.027827537	0.027827498	0.027827403	0.02782725	0.027827132	0.027826663	0.027825404
0.02745364	0.027453649	0.027453635	0.02745363	0.027453624	0.027453606	0.027453575	0.027453498	0.027453373	0.027453278	0.027452897	0.027451876
];

size1 = size(zeroRates,2);

df = zeros(size1,1);
for idx=1:size1

rawData(:,1) = tNodes(:,1);
rawData(:,2) = zeroRates(:,idx);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);


valueDate = H_Date('2023-05-16');
expiryDate = H_Date('2025-03-10');
t1Days = expiryDate.DateDiff(valueDate);
t1 = t1Days/365;


df(idx) = zeroCurve.DF(t1);

end

aaa = 1.0;



AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve

% SOFR zero
rawData = [0.0082191707442756	0.048663423
0.0246575392892821	0.048644353
0.0438356070215719	0.049551895
0.0630137009012237	0.049998172
0.0876712347295253	0.050269984
0.1726027459992920	0.0507055
0.2547945270998130	0.05097079
0.5068493204282380	0.050671672
0.7589041017471120	0.04934487
1.0082191704579900	0.04762846
2.0136986423045600	0.0403535
3.0109589072086500	0.036440234
4.0082191622026800	0.034348878
5.0109589100457900	0.033188526
6.0109589038745600	0.032522882
7.0109589144122900	0.03205825
8.0164383712636300	0.031840422
9.0136986367259400	0.031745499
10.0136986290190000	0.031742999
12.0136986605686000	0.031874385
15.0164383705195000	0.032102624
20.0219178027495000	0.031755804
25.0273972988109000	0.030545653
30.0328767478073000	0.029334334
40.0328767791405000	0.026366547
50.0410959329188000	0.023234369
];


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
AllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface

%SOFR Swaption
% 
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.016724	0.0155	0.014557	0.013685	0.013	0.011963	0.0111	0.0107345	0.010369
2	0.014216	0.0135	0.012972	0.012398	0.011967	0.011167	0.010455	0.0101425	0.00983
3	0.012746	0.012337	0.011951	0.011589	0.011271	0.010636	0.010031	0.0097095	0.009388
4	0.011939	0.011452	0.011134	0.010851	0.010596	0.010043	0.009509	0.009217	0.008925
5	0.011175	0.010758	0.010514	0.010299	0.0101	0.009665	0.009216	0.008911	0.008606
7	0.009947	0.009568	0.009397	0.009254	0.009116	0.008783	0.008431	0.0081535	0.007876
10	0.008545	0.008348	0.008228	0.008118	0.008008	0.007738	0.008545	0.008545	0.008545
15	0.006989	0.006876	0.00677	0.006673	0.006589	0.006431	0.006989	0.006989	0.006989
];



mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
swaptionVolMktData = MktData(mktData);
AllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
annuity = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

% fwdSwapRateLibor = [0.031678775	0.030568683	0.029087482	0.027935757	0.027265993	0.026470368	0.025949941	0.025784802	0.025576224
% 0.029424951	0.027734264	0.026615947	0.026084548	0.025759367	0.025345541	0.025190644	0.025131796	0.024999453
% 0.025996557	0.025154425	0.02491176	0.024782479	0.024675054	0.024594962	0.024673189	0.024680474	0.024586642
% 0.024291077	0.024348953	0.024357482	0.024323931	0.02430362	0.024426209	0.024529579	0.024521072	0.024434692
% 0.02440825	0.024391913	0.024335425	0.024306951	0.024351404	0.024500559	0.024570987	0.024515143	0.024410065
% 0.024218259	0.024217774	0.024322716	0.024482314	0.024547853	0.024612813	0.024543613	0.024457797	0.024059928
% 0.024985251	0.024906987	0.024879675	0.024849918	0.024819235	0.024650568	0.024445241	0.023981298	0.023371462
% 0.024249349	0.024190881	0.024134153	0.0240782	0.024022825	0.023289874	0.02250814	0.02204207	0.021422613
% ];
% 
% annuityLibor = [0.96083198	1.89340341	2.80073986	3.68577888	4.54960754	6.21563388	8.56778834	10.04175615	12.12084134
% 0.93257143	1.83990788	2.72494690	3.58877556	4.43186692	6.05811131	8.35305854	9.79102821	11.82006678
% 0.90733645	1.79237547	2.65620413	3.49929549	4.32223048	5.90953841	8.14835274	9.55127912	11.53190812
% 0.88503902	1.74886769	2.59195904	3.41489403	4.21820343	5.76704848	7.95112033	9.32010149	11.25378216
% 0.86382866	1.70692002	2.52985500	3.33316441	4.11716293	5.62811164	7.75890364	9.09511988	10.98317990
% 0.82293498	1.62624439	2.41024292	3.17508944	3.92119163	5.35916129	7.38819987	8.66182312	10.46541610
% 0.76484652	1.51094871	2.23881433	2.94891837	3.64174071	4.97795695	6.86601697	8.05517318	9.74886255
% 0.67615882	1.33621624	1.98062900	2.60983949	3.22427626	4.41343248	6.10712184	7.18139859	8.71877663
% ];

for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
        annuity(i,j) = KRWBlack.PV01(expiry(i),expiry(i)+tenor(j),0.25);
        swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
    end
end
%% create a CK1FRisky Model with initial modelParams
vol_r.tenor = [1;2;3;4;5;7;10;15];

vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
% meanR_r = 0.03;

meanR_r = 0.01;


joint_swap_tenor = 1;

targetSize = 15;

freq = 4;

matu = DateDiff(H_Date('2037-03-28'),H_Date(valueDate))/365.0;

modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                    {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});

%     import irModels.*;
%     import targetFunctions.*;

KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(AllMktData, modelParams)));

tic
%     out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
%     out = KRWCK1FRisky.CalibrateToATMDiagSwaption(KRWBlack,'global');
out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');

KRWCK1FRisky.vol_r.quote = ones(1,length(KRWCK1FRisky.vol_r.tenor)) * 0.0112140596926117;



% for idx3 = 1:length(out.vol_r.quote)
% KRWVol_r(idx2,idx3) = out.vol_r.quote(idx3);
% KRWRe(idx2,idx3) = out.re(idx3);
% end
% KRWRMSE(idx2) = out.rmseTotal;

toc
%% CK1FRisky model building end
%% SOFR3M RangeAccrual valuation start
%RA info
swaptionPriceSimu = zeros(size(expiry,1),size(tenor,1));
% for i=1:size(expiry,1)
%     for j=1:size(tenor,1)
        
for i=5:5
    for j=5:5
        rangeAccrualParams.payCcy = 'KRW';
        rangeAccrualParams.startDate = H_Date('2022-11-16');
        endDate =  H_Date('2026-11-16');
        
        dateDiff = endDate.DateDiff(H_Date(valueDate));
        
        rangeAccrualParams.maturity = 4;

%         rangeAccrualParams.maturity = expiry(i);

        valueDateH = H_Date(valueDate);

        nominal = 10000;
%         tenorIn = tenor(j);
        tenorIn = 5.0;

%         strike    = fwdSwapRate(i,j);
%         strike    = fwdSwapRateLibor(i,j);
        strike    = 0.027;
        
        
        annuityIn = 1.0;
%         annuityIn = annuityLibor(i,j);

        % strike = 0.0243514043336616;

        %resetDate startDate endDate paymentDate
        couponScheduleInt = [20221116	20221116	20231116	20231116
            20231116	20231116	20241116	20241116
            20241116	20241116	20251116	20251116
            20251116	20251116	20261116	20261116
            ];

        manualScheduleYN = 0;


        rangeAccrualInfo = containers.Map({'nominal','tenor','strike','annuityIn', ...
                                        'couponScheduleInt','manualScheduleYN'}, ...
                                {nominal,tenorIn,strike,annuityIn,...
                               couponScheduleInt,manualScheduleYN});

        rangeAccrualParams.params = rangeAccrualInfo;              

        % rangeAccrual = CDRangeAccrual(rangeAccrualParams);
        rangeAccrual = Swaption(rangeAccrualParams);



        numOfRandom = 40000;
%         numOfRandom = 200000;
        
        numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                                {'AMC',1,numOfRandom});

        numMethodAMC = NumMethod(numMethodInfo);

        % weekly step calculator
        % pricer = Pricer(rangeAccrual,USDLGM,numMethodAMC);
        % pricingOut = pricer.computePrice(valueDate);

        % daily weekly step calculator
        % pricer = DailyWeeklyRiskyPricer(rangeAccrual,USDLGMRisky,numMethodAMC,true);
        % pricer = DailyWeeklyRiskyPricer_CDRA(rangeAccrual,KRWCK1FRisky,numMethodAMC,false);

        pricer = DailyWeeklyRiskyPricer_Swaption(rangeAccrual,KRWCK1FRisky,numMethodAMC,false);
%         pricingOutMC = pricer.computePriceMCCallable(valueDateH,'PSA');
%         pricingOutMC = pricer.computePriceMCCallable(valueDateH,'PSAFallback');
        
%         pricingOutMC = pricer.computePriceMCCallableStochAnnuity(valueDateH,'Fallback');
        closedFormPayerSwaption_ck1f = KRWCK1FRisky.payerSwaption(dateDiff/365.0,tenorIn,strike);
        
        closedFormStraddleSwaption_ck1f = KRWCK1FRisky.straddleSwaption(dateDiff/365.0,tenorIn,strike);
        
        
        
        pricingOutMC0 = pricer.computePriceMCCallableStochAnnuity(valueDateH,'NoFallback');
        swaptionPriceSimuSOFRMC = pricingOutMC0.nonCall.npv;
        swaptionPriceSimuSOFRMCStrad = pricingOutMC0.straddle.npv;
        
        pricingOutMC1 = pricer.computePriceMCCallableStochAnnuity(valueDateH,'Fallback');
        swaptionPriceSimufallbackMC = pricingOutMC1.nonCall.npv;
        swaptionPriceSimufallbackMCStrad = pricingOutMC1.straddle.npv;
        
        pricingOutMC2 = pricer.computePriceMCCallableStochAnnuity(valueDateH,'SemiAnnual_Fallback');
        swaptionPriceSimuLiborSwaptionfallbackMC = pricingOutMC2.nonCall.npv;
        swaptionPriceSimuLiborSwaptionfallbackMCStrad = pricingOutMC2.straddle.npv;
        
        pricingOutMC3 = pricer.computePriceMCCallableStochAnnuity(valueDateH,'SemiAnnual_Fallback2');
        
        swaptionPriceSimuLiborSwaptionfallbackMC2 = pricingOutMC3.nonCall.npv;
        swaptionPriceSimuLiborSwaptionfallbackMCStrad2 = pricingOutMC3.straddle.npv;
        
        pricingOutMC4 = pricer.computePriceMCCallableStochAnnuity(valueDateH,'SemiAnnual_Fallback_henrard');
        
        swaptionPriceSimuLiborSwaptionfallbackMC3 = pricingOutMC4.nonCall.npv;
        swaptionPriceSimuLiborSwaptionfallbackMCStrad3 = pricingOutMC4.straddle.npv;
        
%         pricingOutMC = pricer.computePriceMCCallableStochAnnuity(valueDateH,'SemiAnnual_Fallback');
        
%         pricingOutMC = pricer.computePriceMCCallableStochAnnuity(valueDateH,'NoFallback');
        
%         pricingOutMC = pricer.computePriceMCCallable(valueDateH,'PSAFallbackSimple');
        
        
%         swaptionPriceSimu(i,j) = pricingOutMC.nonCall.npv;
        aaa = 1.0;
    end
end
% pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);

% daily closed form pricer
%pricingOutLGM =  rangeAccrual.computePrice(valueDate,USDLGM);

%pricingOut = rangeAccrual.computePriceMCForward(valueDate,USDLGM);

% % out =1;
% newexpiry = [1;2;3;4;5;7;10;15];
% 
% 
% testCK.impCorrel = zeros(size(newexpiry,1),size(newexpiry,1));
% testCK.InsSwapFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));
% testCK.impFwdLiborCorr = USDLGM.InsFwdCorrTest2(0,newexpiry,0.25);
% for i=1:size(newexpiry,1)
%     for j=1:size(newexpiry,1)
%         testCK.impCorrel(i,j)= USDLGM.ImpCorrel(5,newexpiry(i),newexpiry(j));
%         testCK.InsSwapFwdCorr(i,j) = USDLGM.InsSwapFwdCorrTest(5,newexpiry(i),newexpiry(j));
%     end
% end
% 
% newexpiryFwd = USDLGM.dH1.tenor;
% testCK.impFwdLiborCorrNew = USDLGM.InsFwdCorrTest2(0,newexpiryFwd,0.25);
% 
% 
% newexpiry2 = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
% 
% for i=1:size(newexpiry2,1)
%     testCK.impCorrel10_2(i)= USDLGM.ImpCorrel(newexpiry2(i),10,2);
%     testCK.impCovarl10_2(i)= sqrt(USDLGM.ImpCovar(newexpiry2(i),10,2)/newexpiry2(i));
%     testCK.spreadInfo10_2(i) =  USDLGM.SpreadInfo(newexpiry2(i),10,2);
%     testCK.fwdSpread(i) = testCK.spreadInfo10_2(i).fwdSpread';
%     testCK.muSpread(i) = testCK.spreadInfo10_2(i).mu';
%     testCK.sigma(i) = testCK.spreadInfo10_2(i).sigma';
%     testCK.corr(i) = testCK.spreadInfo10_2(i).corr;
%     testCK.mu1(i) =testCK.spreadInfo10_2(i).mu1;
%     testCK.mu2(i) =testCK.spreadInfo10_2(i).mu2;
%     testCK.sig1(i) =testCK.spreadInfo10_2(i).sig1;
%     testCK.sig2(i) =testCK.spreadInfo10_2(i).sig2;
%     
% end
% 
aaaa =1.0;


In [ ]:
%test_anderson_hong.m

function test_anderson(m,deltas)
%test_anderson    Test Anderson acceleration for NCM problem.
%   test_anderson(M, DELTAS) computes the nearest correlation matrix with
%   smallest eigenvalue at least DELTAS(i) for two
%   invalid correlation test matrices by the unaccelerated alternating
%   projections method, nearcorr_new, and by alternating
%   projections with Anderson acceleration, nearcorr_aa, with 
%   history length M.
%   Vector DELTAS specifies different values of the lower bound
%   delta on the smallest eigenvalue of the solution.
%   Default: M = 2, DELTAS = [0 1e-8 0.1].

if nargin < 1, m = 2; end
if nargin < 2, deltas = [0 1e-8 0.1]; end

f = get(0,'Format'); % Save current format.
format shortg, format compact

fprintf('*** Test matrix 1:\n')
% Turkay, Epperlein, and Christofides (2003).
A = [1 -0.55 -0.15 -0.10;
    -0.55 1 0.90 0.90;
    -0.15 0.90 1 0.90;
    -0.10 0.90 0.90 1]

pattern = [];

for delta = deltas
    fprintf('--> m = %d, delta = %d\n', m, delta)
    [X,it] = nearcorr_new(A,pattern,delta);
    [XAA,itAA] = nearcorr_aa(A,pattern,m,100,'u',delta); 
    fprintf(['Iterations for NCM: standard = %d, accelerated = %d.' ...
             '  Reduction factor = %.2f\n'], it, itAA, it/itAA)

end

fprintf('\n*** Test matrix 2 and the pattern of elements to keep fixed:\n')
% Finger (1997).
A = [1	0.68249032	0.263832649	0.268129694	0.065572083	0.065572083	0.090456492	0.090456492;
0.68249032	1	0.323298733	0.393147389	0.075601255	0.075601255	0.091235355	0.091235355;
0.263832649	0.323298733	1	0.660784665	0.14968895	0.14968895	0.156980229	0.156980229;
0.268129694	0.393147389	0.660784665	1	0.251866048	0.251866048	0.25665452	0.25665452;
0.065572083	0.075601255	0.14968895	0.251866048	1	0	0.9999	0;
0.065572083	0.075601255	0.14968895	0.251866048	0	1	0	0.9999;
0.090456492	0.091235355	0.156980229	0.25665452	0.9999	0	1	0;
0.090456492	0.091235355	0.156980229	0.25665452	0	0.9999	0	1;
]

% A= []

% Define pattern to keep fixed
% pattern = blkdiag(ones(3,3),eye(4));
pattern = [1	0	0	0	0	0	0	0
0	1	0	0	0	0	0	0
0	0	1	0	0	0	0	0
0	0	0	1	0	0	0	0
0	0	0	0	1	1	0	1
0	0	0	0	1	1	1	0
0	0	0	0	0	1	1	1
0	0	0	0	1	0	1	1
]


for delta = deltas
    fprintf('--> m = %d, delta = %d\n', m, delta)
    [X,it] = nearcorr_new(A,pattern,delta);
    [XAA,itAA] = nearcorr_aa(A,pattern,m,100,'u',delta); 
    fprintf(['Iterations for NCM: standard = %d, accelerated = %d.' ...
             '  Reduction factor = %.2f\n'], it, itAA, it/itAA)

end

format(f) % Restore original format.


In [ ]:
function test_anderson(m,deltas)
%test_anderson    Test Anderson acceleration for NCM problem.
%   test_anderson(M, DELTAS) computes the nearest correlation matrix with
%   smallest eigenvalue at least DELTAS(i) for two
%   invalid correlation test matrices by the unaccelerated alternating
%   projections method, nearcorr_new, and by alternating
%   projections with Anderson acceleration, nearcorr_aa, with 
%   history length M.
%   Vector DELTAS specifies different values of the lower bound
%   delta on the smallest eigenvalue of the solution.
%   Default: M = 2, DELTAS = [0 1e-8 0.1].

if nargin < 1, m = 2; end
if nargin < 2, deltas = [0 1e-8 0.1]; end

f = get(0,'Format'); % Save current format.
format shortg, format compact

fprintf('*** Test matrix 1:\n')
% Turkay, Epperlein, and Christofides (2003).
A = [1 -0.55 -0.15 -0.10;
    -0.55 1 0.90 0.90;
    -0.15 0.90 1 0.90;
    -0.10 0.90 0.90 1]

pattern = [];

for delta = deltas
    fprintf('--> m = %d, delta = %d\n', m, delta)
    [X,it] = nearcorr_new(A,pattern,delta);
    [XAA,itAA] = nearcorr_aa(A,pattern,m,100,'u',delta); 
    fprintf(['Iterations for NCM: standard = %d, accelerated = %d.' ...
             '  Reduction factor = %.2f\n'], it, itAA, it/itAA)

end

fprintf('\n*** Test matrix 2 and the pattern of elements to keep fixed:\n')
% Finger (1997).
A = [1 0.18 -0.13 -0.26 0.19 -0.25 -0.12;
    0.18 1 0.22 -0.14 0.31 0.16 0.09;
   -0.13 0.22 1 0.06 -0.08 0.04 0.04;
   -0.26 -0.14 0.06 1 0.85 0.85 0.85;
    0.19 0.31 -0.08 0.85 1 0.85 0.85;
   -0.25 0.16 0.04 0.85 0.85 1 0.85;
   -0.12 0.09 0.04 0.85 0.85 0.85 1]

% Define pattern to keep fixed
pattern = blkdiag(ones(3,3),eye(4))

for delta = deltas
    fprintf('--> m = %d, delta = %d\n', m, delta)
    [X,it] = nearcorr_new(A,pattern,delta);
    [XAA,itAA] = nearcorr_aa(A,pattern,m,100,'u',delta); 
    fprintf(['Iterations for NCM: standard = %d, accelerated = %d.' ...
             '  Reduction factor = %.2f\n'], it, itAA, it/itAA)

end

format(f) % Restore original format.


In [ ]:
%test_A_rg.m

%TEST   Simple test of EXPMV_TSPAN.
clc;
clear;

n = 50;
A = -gallery('poisson',n);
b = linspace(-1,1,n^2)';

load('A_rg');
load('lastProb');
dT = 0.060273972602740;

for idxH=1:100
tic
newProbT = expmv(dT,A_rg',lastProb');
toc
newProb =newProbT';

tic
A_exp1 = expm(A_rg*dT);
newProb1 = lastProb*A_exp1;
toc

diffA = newProb-newProb1;
normA = norm(diffA);
fprintf('diff norm is %4.2f \n',normA);

end

t0 = 0; tmax = 1;
q = 9;

tic
[X,tvals,mv] = expmv_tspan(A,b,t0,tmax,q);
toc

fprintf('Relative differences between vectors from EXPM and EXPMV_TSPAN.\n')
fprintf('Should be of order %9.2e.\n', eps/2)
Y = zeros(size(X));
for i = 1:length(tvals)
    tic
    Y(:,i) = expm(full(tvals(i)*A))*b;
    toc
    fprintf('%2.0f:  %9.2e\n', i, norm(Y(:,i)-X(:,i),1)/norm(X(:,i),1) )
end

Y1 = zeros(size(X));
for i = 1:length(tvals)
    tic
    Y1(:,i) = expmv(tvals(i),A,b);
    toc
    fprintf('%2.0f:  %9.2e\n', i, norm(Y1(:,i)-X(:,i),1)/norm(X(:,i),1) )
end



In [ ]:
%test_20211029_virtual_irs_20230103.m


clc
clear
tic

% DF asof settleDate 
valueDate = '2021-10-29';

% valueDate = '2021-11-02';

% useSOFRAsDiscountCurve = true;
useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;



for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({GBENCalendar,USFDCalendar});


% %% import and construct discount curve: USD IRS(Libor3M) Curve
%% import and construct discount curve: USD SOFR Curve
if ~useSOFRAsDiscountCurve

    dataType = {
    'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Deposit'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    'Swap'
    };

    dataTenor = {
    'SN'
    '1M'
    '2M'
    '3M'
    '6M'
    '1Y'
    '2Y'
    '3Y'
    '4Y'
    '5Y'
    '7Y'
    '10Y'
    '12Y'
    '15Y'
    '20Y'
    '25Y'
    '30Y'
    '50Y'
    };

    dataRate = {
    0.00072130
0.00087500
0.00102630
0.00132250
0.00201000
0.00313800
0.00693400
0.00968000
0.01126600
0.01234700
0.01413800
0.01568400
0.01631200
0.01691300
0.01740000
0.01740100
0.01725700
0.01563700
    };

% % virtual IRS
% dataRate = {
%     0.0005643999999633080
% 0.0015867999999898400
% 0.0023074945876241500
% 0.0031030999999946200
% 0.0049596224427244200
% 0.0043888999340078400
% 0.0077764675494812700
% 0.0101782731943017000
% 0.0115858908938469000
% 0.0125682466894138000
% 0.0142090856753550000
% 0.0156278108637616000
% 0.0162174916218490000
% 0.0167861264857747000
% 0.0172582101336186000
% 0.0172492664180966000
% 0.0170902604088566000
% 0.0154458003303209000
%     };


% % virtual IRS2
% dataRate = {
%     0.00072130
% 0.00087500
% 0.00102630
% 0.00132250
% 0.00201000
% 0.00313800
% 0.0077764675494812700
% 0.0101782731943017000
% 0.0115858908938469000
% 0.0125682466894138000
% 0.0142090856753550000
% 0.0156278108637616000
% 0.0162174916218490000
% 0.0167861264857747000
% 0.0172582101336186000
% 0.0172492664180966000
% 0.0170902604088566000
% 0.0154458003303209000
%     };
else

    dataType= {
    'Deposit'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    'OIS'
    };

    dataTenor = {
    'SN'
    '1W'
    '2W'
    '3W'
    '1M'
    '2M'
    '3M'
    '6M'
    '12M'
    '2Y'
    '3Y'
    '4Y'
    '5Y'
    '6Y'
    '7Y'
    '8Y'
    '9Y'
    '10Y'
    '12Y'
    '15Y'
    '20Y'
    '25Y'
    '30Y'
    '40Y'
    '50Y'
    };

    dataRate = {
    0.00050000
0.00042100
0.00042100
0.00044500
0.00044200
0.00046100
0.00048700
0.00067100
0.00177000
0.00516000
0.00756700
0.00897900
0.00996500
0.01086800
0.01161300
0.01218800
0.01264500
0.01303900
0.01363200
0.01420400
0.01467900
0.01467000
0.01451000
0.01377000
0.01285600
    };

end

% construct iborIndex
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

if ~useSOFRAsDiscountCurve
    
    IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'3M','USD',2,GBENCalendar,'Following','Act360',false,dummyCurve});
else                    
                    
    IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,dummyCurve});
                    
end

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;
IborIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;


if ~useSOFRAsDiscountCurve
    %% index fixing for Libor3M start
    tempDate = H_Date('2021-10-29');
    irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.00132250});
    %% index fixing for libor3M_index end
    IborIndexParams('historicalFixings') = irIndexFixing1;
end

% overnight index itself is nothing but 1D IborIndex
if ~useSOFRAsDiscountCurve
    libor3M_Index = IborIndex(IborIndexParams);
else
    overnightIndex = IborIndex(IborIndexParams);
end

rateHelpers = {};

for i=1:length(dataRate)
    if strcmp(dataType{i},'Deposit')
        depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
                        {IborIndexParams});

        calcCalendar = GBENCalendar;
        
        depositRateHelperParams('calcCalendar') = calcCalendar;
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
        spotLag = 2;
        
        depositRateHelperParams('spotLag') = spotLag;
        
        depositRateHelperParams('rate') = dataRate{i};
        depositRateHelperParams('rateTenor') = dataTenor{i};
        depositRateHelper = DepositRateHelper(depositRateHelperParams);
    
        rateHelpers{end+1} = depositRateHelper;
        
    elseif strcmp(dataType{i},'OIS')
        
        if i== 17
            bbb = 1.0;
        end
        % cashflow interval(?)
        fixedLegTenor = '1Y';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = 'Act360';
        
        floatingLegTenor = '1Y';
        floatingLegConvention = 'ModifiedFollowing';
        floatingLegDayCounter = 'Act360';
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
        spotLag = 2;
        
        % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
        
%         payLag  = 2;
        payLag  = 2;

        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
        
        calcCalendar = USFDCalendar;

        oisRateHelperParams = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter',...
                                'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','overnightIndex','discountCurve'}, ...
                                {calcCalendar,spotLag,payLag, fixedLegTenor,fixedLegConvention,fixedLegDayCounter,...
                                 floatingLegTenor,floatingLegConvention,floatingLegDayCounter,overnightIndex,dummyCurve});

%         oisRateHelperParams('useRounding') = useRounding;
        
        oisRateHelperParams('rate') = dataRate{i};
        oisRateHelperParams('rateTenor') = dataTenor{i};
        oisRateHelper = OISRateHelper(oisRateHelperParams);
        rateHelpers{end+1} = oisRateHelper;
        
    elseif strcmp(dataType{i},'Swap')
        fixedLegTenor = '6M';
        fixedLegConvention = 'ModifiedFollowing';
        fixedLegDayCounter = '30360';
        
        floatingLegTenor = '3M';
        floatingLegConvention = 'ModifiedFollowing';
        floatingLegDayCounter = 'Act360';
        
        % spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
        spotLag = 2;
        
        % payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D
        
%         payLag  = 2;
        payLag  = 0;
        
        if i== 5
            bbb = 1.0;
        end

        calcCalendar = jointCalendar_USDIRS;
        
        %discountCurve
        
        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        swapRateHelperParams = containers.Map({'calcCalendar','spotLag','payLag','fixedLegTenor','fixedLegConvention','fixedLegDayCounter', ...
                                                'floatingLegTenor','floatingLegConvention','floatingLegDayCounter','IborIndex','discountCurve'}, ...
                                {calcCalendar,spotLag,payLag,fixedLegTenor,fixedLegConvention,fixedLegDayCounter, ...
                                 floatingLegTenor,floatingLegConvention,floatingLegDayCounter,libor3M_Index,dummyCurve});


        swapRateHelperParams('rate') = dataRate{i};
        swapRateHelperParams('rateTenor') = dataTenor{i};
        swapRateHelper = SwapRateHelper(swapRateHelperParams);
        rateHelpers{end+1} = swapRateHelper;
    end
end

% forward & discount zeroCurve
dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','forwardCurve','discountCurve'},{rateHelpers,dummyCurve,dummyCurve1});

% tic
% yieldCurve = YieldCurve(yieldCurveParams);
yieldCurve = YieldCurveOIS(yieldCurveParams);

if ~useSOFRAsDiscountCurve
    outSOFR = yieldCurve.BootStrap();
else
%     out = yieldCurve.BootStrap();
    outSOFR = yieldCurve.GlobalCalib();
end
toc
ddd = 1.0;

targetTimeBucket=[
0.002739726
0.083333333
0.166666667
0.25
0.5
0.75
1
1.5
2
2.5
3
3.5
4
4.5
5
5.5
6
6.5
7
7.5
8
8.5
9
9.5
10
10.5
11
11.5
12
12.5
13
13.5
14
14.5
15
15.5
16
16.5
17
17.5
18
18.5
19
19.5
20
20.5
21
21.5
22
22.5
23
23.5
24
24.5
25
25.5
26
26.5
27
27.5
28
28.5
29
29.5
30
30.5
31
31.5
32
32.5
33
33.5
34
34.5
35
35.5
36
36.5
37
37.5
38
38.5
39
39.5
40
40.5
41
41.5
42
42.5
43
43.5
44
44.5
45
45.5
46
46.5
47
47.5
];

liborZero = zeros(length(targetTimeBucket),1);
for idxjj=1:length(targetTimeBucket)
liborZero(idxjj) = yieldCurve.forwardCurve.Zero(targetTimeBucket(idxjj));
end
fff = 1.0;
%% discount curve bootstrapping end 

%% SOFR Compounding + Spread 1Period OIS(Or Libor Equivalent Deposit)

%% index fixing for SOFR Index start
tempDate = H_Date('2021-10-29');
irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0005});
for idxhong=1:10000
    tempDate = AddDate(tempDate,1,'day');
    irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0005;
end
%% index fixing for SOFR_index end

% % %% index fixing for Libor3M start
% % tempDate = H_Date('2021-09-24');
% % irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.001322500});
% % for idxhong=1:10000
% %     tempDate = AddDate(tempDate,1,'day');
% %     irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.001322500;
% % end
% %% index fixing for Libor3M end

% %% index fixing for Libor6M start
% tempDate = H_Date('2021-09-24');
% irIndexFixing_Libor6M = containers.Map({StrDate(tempDate,'str')},{0.00155380});
% for idxhong=1:10000
%     tempDate = AddDate(tempDate,1,'day');
%     irIndexFixing_Libor6M(StrDate(tempDate,'str')) = 0.00155380;
% end
% %% index fixing for Libor6M end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);


valueDateH = H_Date('2021-10-29');

% SOFR +Spread vs Fixed OIS(Libor Equivalent Deposit) 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = outSOFR.forwardCurve.params('rawData');

% rawData_SOFR = [
% 0.002739726027	0.000101388875
% 0.019178082192	0.000192790616
% 0.038356164384	0.000188582651
% 0.057534246575	0.000190610066
% 0.087671232877	0.000186047094
% 0.167123287671	0.000282614854
% 0.252054794521	0.000344707247
% 0.339726027397	0.000367004898
% 0.419178082192	0.000400452499
% 0.509589041096	0.000415650422
% 0.586301369863	0.000427300637
% 0.671232876712	0.000435908443
% 0.758904109589	0.000446542384
% 0.835616438356	0.000470858745
% 0.917808219178	0.000496692325
% 1.000000000000	0.000516949692
% 1.506849315068	0.000720697544
% 2.005479452055	0.001120059319
% 3.002739726027	0.002714418174
% 4.002739726027	0.004933516134
% 5.002739726027	0.007049886611
% 6.002739726027	0.008952079387
% 7.010958904110	0.010559045064
% 8.008219178082	0.011830012220
% 9.005479452055	0.012885165460
% 10.005479452055	0.013741279433
% 12.008219178082	0.015097592964
% 15.010958904110	0.016378720092
% 20.013698630137	0.017426478939
% 25.019178082192	0.017726708805
% 30.024657534247	0.017768042382
% 40.027397260274	0.016871633528
% 50.032876712329	0.015711420383
% ];

% rawData_Libor3M_DC = [
% 0.087671232876712	0.001098799851025
% 0.167123287671233	0.001492765168950
% 0.252054794520548	0.001777759972902
% 0.509589041095890	0.002093881108158
% 1.000000000000000	0.002082797416369
% 2.005479452054790	0.002757637808099
% 3.005479452054790	0.004599469233654
% 4.002739726027400	0.006983466216479
% 5.002739726027400	0.009201864211354
% 7.010958904109590	0.012826605809096
% 10.005479452054800	0.016117892407056
% 12.008219178082200	0.017531896005676
% 15.010958904109600	0.018870232807240
% 20.016438356164400	0.019987631397089
% 25.021917808219200	0.020322045675130
% 30.024657534246600	0.020377680903766
% 50.032876712328800	0.018354038457619
% ];

% rawData_Libor3M_SC = [
% 0.08219178082192	0.00086309299659
% 0.16986301369863	0.00107969265373
% 0.25205479452055	0.00134064151789
% 0.49589041095890	0.00157476551976
% 1.00000000000000	0.00181389161153
% 2.00000000000000	0.00388289997209
% 3.00821917808219	0.00666973501502
% 4.00547945205480	0.00886687150632
% 5.00273972602740	0.01053286170063
% 7.00547945205480	0.01289464819420
% 10.00821917808220	0.01499969183351
% 12.00821917808220	0.01591323247994
% 15.01369863013700	0.01680581056426
% 20.01917808219180	0.01759287586028
% 25.01643835616440	0.01771894345875
% 30.01917808219180	0.01762506722589
% 50.03287671232880	0.01568648774992
% ];

% rawData_Libor3M_SC = [
% 0.25205479452055	0.00134064151849
% 0.46301369863014	0.00162632687055
% 0.71232876712329	0.00160291561001
% 0.98082191780822	0.00172653975112
% 1.23013698630137	0.00204677691416
% 1.46027397260274	0.00254611438684
% 1.72876712328767	0.00316952715072
% 2.00000000000000	0.00388324457455
% 3.00821917808219	0.00667012898028
% 4.00547945205479	0.00886726666610
% 5.00273972602740	0.01053323980812
% 6.00273972602740	0.01184156282527
% 7.00547945205479	0.01289355558312
% 8.00547945205479	0.01372541315620
% 9.01095890410959	0.01440845793027
% 10.00821917808220	0.01499572035007
% 11.00821917808220	0.01548567817170
% 12.00821917808220	0.01590921338321
% 15.01369863013700	0.01680238167095
% 20.01917808219180	0.01759015717092
% 25.01643835616440	0.01771674414010
% 30.01917808219180	0.01762324243026
% 40.02739726027400	0.01676238185267
% 50.03287671232880	0.01567119485148
% ];
% 
% rawData_Libor6M_SC = [
% 0.495890410958904	0.00157476551959538
% 1.000000000000000	0.00190320852104560
% 1.495890410958900	0.00281881092529045
% 2.000000000000000	0.00414799207446455
% 3.008219178082190	0.00739411429421069
% 4.005479452054790	0.00981315646575917
% 5.002739726027400	0.01160763809543230
% 7.005479452054790	0.01411636225359480
% 10.008219178082200	0.01632472746632380
% 12.008219178082200	0.01727125337602080
% 15.013698630137000	0.01821074304712790
% 20.019178082191800	0.01903904283053590
% 25.016438356164400	0.01920537065227420
% 30.019178082191800	0.01913460269994890
% ];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

% IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useSimpleTeleScoping') = true;

IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

% %% Libor3M Index
% IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});
% 
% 
% IborIndexParams_fwd('forecastTodaysFixing') = true;
% IborIndexParams_fwd('useRounding') = useRounding;
% IborIndexParams_fwd('useSimpleTeleScoping') = false;
% IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
% IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;
% 
% libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);
% 
% %% Index Definition End


referenceDate = H_Date('2021-10-29');

% basis swap's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

targetTenor = {
'1D'
'1M'
'2M'
'3M'
'6M'
};

targetSpread = {
0.0000644000
0.0011448
0.0018456
0.0026161
0.0042826
};

% targetTenor = {
% '1D'
% '1M'
% '2M'
% '3M'
% '6M'
% };
% 
% targetSpread = {
% 0.0000644000
% 0.0011448
% 0.0018456
% 0.0026161
% 0.0042826
% };

simulNPV = zeros(100,1);
simulFairRate = zeros(100,1);

for idxhong=1:length(targetSpread)

rateTenor = targetTenor{idxhong};
rateData = targetSpread{idxhong};

% SOFR OIS 
fixedLegTenor = '1Y';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = 'Act360';

floatingLegTenor = '1Y';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';

% spotLag : effectiveDate of Swap contract: effectiveDate = asOfDate + 2D
spotLag = 2;

% payLag : paymentDate Coupon of Legs of Swap:  paymentDate = endDate + 2D

%         payLag  = 2;
payLag  = 0;
        
startDate = calcCalendar.Advance(referenceDate,overnightIndex.convention,...
                                    spotLag,'day',overnightIndex.endOfMonth);

                                % 
endDate = AddTenor(startDate,rateTenor);


useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end


% fixed & floating schedule 

fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor ,calcCalendar,...
                    fixedLegConvention,fixedLegConvention, 'Backward',useEndOfMonth); 

floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                    floatingLegConvention,fixedLegConvention, 'Backward',useEndOfMonth);

type = 'Payer';
nominal = 1.0;
fixedRate = 0.0;
fixedDayCounter = fixedLegDayCounter;
floatingSpread = rateData;
floatingDayCounter = floatingLegDayCounter;
vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
                      floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);

outVanillaOIS = vanillaSwap.Calculate(discountCurve_SOFR);
             
simulFairRate(idxhong) = outVanillaOIS.fairRate;
simulNPV(idxhong) = outVanillaOIS.NPV;

end

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;



In [ ]:
function out = TargetStrikeFromDeltaNormalSABRReduced(delta,term,alpha,rho,nu)
%  compute targetStrike From delta
%  using normalSABR interpolated vol strip
    if delta <= 0
        p = norminv(-1.0*delta);
        vol1 = @(strike) SABRNormalVol(0.0,strike,term,alpha,rho,nu);
        targetFunc = @(x) vol1(x)*sqrt(term)*p - x;
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[-1000 1000]);
        out = solutionK;
        
    else
        p = norminv(1.0*delta);
        vol1 = @(strike) SABRNormalVol(0.0,strike,term,alpha,rho,nu);
        targetFunc = @(x) -vol1(x)*sqrt(term)*p - x;
        [solutionK,fval,exitflag,output] = fzero(targetFunc,[-1000 1000]);
        out = solutionK;
    end
end



In [ ]:
function  out  = TargetFunctionYieldCurveStripGlobal(tvar,yieldCurve,optParams)

    % initialize zeroCurve
    modelParamsSize = optParams.modelParamsSize;
    for i=1:modelParamsSize
        yieldCurve.rateHelpers{i}.Update(i,tvar(i));
    end
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    for i=1:optParams.numOfEquation
        out(idx) = yieldCurve.rateHelpers{i}.QuoteError();
        idx = idx + 1; 
    end
    
end





In [ ]:
function  out  = TargetFunctionVolProxyLevMar1(tvar,eqDupire,optParams)
    idxNow = optParams.idxNow;
    if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
    mkt_dT=mkt_dT/365.0;
    %filling vol proxy grid with the tvar
  %% Solve  Forward PDE One time
    [newC newP] = eqDupire.Solve1DPDE(tvar,mkt_dT,optParams.dK,optParams.lastC,optParams.lastP,optParams);
    optParams.meshC = newC;
    optParams.meshP = newP;
   %% PDE Solve End
    target = CalculateTarget(eqDupire,idxNow,newC,newP,optParams);
    out = target.re;
end



In [ ]:
function  out  = TargetFunctionSSSABRStrip3(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    idxNow = optParams.idxNow;
    % assigning model parameters
    eqCOMDupireSpotGF.alpha_term(idxNow,2) = tvar(1);
    eqCOMDupireSpotGF.rho_term(idxNow,2) = tvar(2);
    eqCOMDupireSpotGF.nu_term(idxNow,2) = tvar(3);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=idxNow:idxNow
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVolStrip(1.0,marketStrike,expiry(i)/365.0,i);
                    model_vol = dummyOut.vol;
                
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=idxNow:idxNow
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=idxNow:idxNow
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=idxNow:idxNow
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal7Term(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    alpha_term_size = size(eqCOMDupireSpotGF.alpha_term,1);
    
    for idxhong=1:alpha_term_size
        eqCOMDupireSpotGF.alpha_term(idxhong,2) = sqrt(tvar(idxhong)*tvar(idxhong));
    end
    
    eqCOMDupireSpotGF.rho = tvar(alpha_term_size+1);
    eqCOMDupireSpotGF.nu = tvar(alpha_term_size+2);
    
    eqCOMDupireSpotGF.lambda = tvar(alpha_term_size + 3);
    eqCOMDupireSpotGF.omega = tvar(alpha_term_size + 4);
    eqCOMDupireSpotGF.time_shift = tvar(alpha_term_size + 5);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal7bc(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    eqCOMDupireSpotGF.lambda = tvar(5);
    eqCOMDupireSpotGF.omega = tvar(6);
    eqCOMDupireSpotGF.time_shift = tvar(7);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal7(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    eqCOMDupireSpotGF.lambda = tvar(5);
    eqCOMDupireSpotGF.omega = tvar(6);
    eqCOMDupireSpotGF.time_shift = tvar(7);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal6Term(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    alpha_term_size = size(eqCOMDupireSpotGF.alpha_term,1);
    
    for idxhong=1:alpha_term_size
        eqCOMDupireSpotGF.alpha_term(idxhong,2) = sqrt(tvar(idxhong)*tvar(idxhong));
    end
    
    eqCOMDupireSpotGF.rho = tvar(alpha_term_size+1);
    eqCOMDupireSpotGF.nu = tvar(alpha_term_size+2);
    
    eqCOMDupireSpotGF.lambda = tvar(alpha_term_size + 3);
    eqCOMDupireSpotGF.omega = tvar(alpha_term_size + 4);
%     eqCOMDupireSpotGF.time_shift = tvar(7);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal6bc(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    eqCOMDupireSpotGF.lambda = tvar(5);
    eqCOMDupireSpotGF.omega = tvar(6);
%     eqCOMDupireSpotGF.time_shift = tvar(7);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.FwdWithoutSpot(expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                    market_otmPrice = optParams.params.blackOTMPrices(i,j);
                    if marketStrike <= 1.0
                        model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
                    else
                        model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
                    end
                    
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal6(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    eqCOMDupireSpotGF.lambda = tvar(5);
    eqCOMDupireSpotGF.omega = tvar(6);
%     eqCOMDupireSpotGF.time_shift = tvar(7);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal4Term(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqCOMDupireSpotGF.alpha_0 = tvar(1);
%     eqCOMDupireSpotGF.alpha_inf = tvar(2);
    
    alpha_term_size = size(eqCOMDupireSpotGF.alpha_term,1);
    
    for idxhong=1:alpha_term_size
        eqCOMDupireSpotGF.alpha_term(idxhong,2) = sqrt(tvar(idxhong)*tvar(idxhong));
    end
    
    eqCOMDupireSpotGF.rho = tvar(alpha_term_size+1);
    eqCOMDupireSpotGF.nu = tvar(alpha_term_size+2);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal4RhoSS(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho_SS = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                    model_vol = dummyOut.vol;
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal4bc(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal4(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    dummyOut = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
                    model_vol = dummyOut.vol;
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionSSSABRGlobal(tvar,eqCOMDupireSpotGF,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqCOMDupireSpotGF.alpha_0 = tvar(1);
    eqCOMDupireSpotGF.alpha_inf = tvar(2);
    eqCOMDupireSpotGF.rho = tvar(3);
    eqCOMDupireSpotGF.nu = tvar(4);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
%            fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrike(i,j);
                    model_vol = eqCOMDupireSpotGF.SABR_ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionsFuturesCorrLevGlobalFixedRho(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    targetTenors = params.params.targetTenors;
    targetCorr = params.params.targetCorr;
    % assigning model parameters
    eqCOMDupireSpotGF.alpha = tvar(1);
    eqCOMDupireSpotGF.beta  = tvar(2);
%     eqCOMDupireSpotGF.rho   = tvar(2);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(targetTenors);
    numOfEquation = expirySize*(expirySize-1)/2;
    out= zeros(numOfEquation,1);
    
    modelCorr = zeros(expirySize,expirySize);
    
    idx =1;
    
    for j=1:expirySize-1
        for k=j+1:expirySize
            modelCorr(j,k) = eqCOMDupireSpotGF.computeFuturesCorr(0.0,targetTenors(j),targetTenors(k));
            out(idx) = 1.0*(modelCorr(j,k)-targetCorr(j,k));
            idx = idx + 1;
        end
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsFuturesCorrLevGlobalFixedBeta(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    targetTenors = params.params.targetTenors;
    targetCorr = params.params.targetCorr;
    % assigning model parameters
    eqCOMDupireSpotGF.alpha = tvar(1);
%     eqCOMDupireSpotGF.beta  = tvar(1);
    eqCOMDupireSpotGF.rho   = tvar(2);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(targetTenors);
    numOfEquation = expirySize*(expirySize-1)/2;
    out= zeros(numOfEquation,1);
    
    modelCorr = zeros(expirySize,expirySize);
    
    idx =1;
    
    for j=1:expirySize-1
        for k=j+1:expirySize
            modelCorr(j,k) = eqCOMDupireSpotGF.computeFuturesCorr(0.0,targetTenors(j),targetTenors(k));
            out(idx) = 1.0*(modelCorr(j,k)-targetCorr(j,k));
            idx = idx + 1;
        end
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsFuturesCorrLevGlobalFixedAlpha(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    targetTenors = params.params.targetTenors;
    targetCorr = params.params.targetCorr;
    % assigning model parameters
%     eqCOMDupireSpotGF.alpha = tvar(1);
    eqCOMDupireSpotGF.beta  = tvar(1);
    eqCOMDupireSpotGF.rho   = tvar(2);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(targetTenors);
    numOfEquation = expirySize*(expirySize-1)/2;
    out= zeros(numOfEquation,1);
    
    modelCorr = zeros(expirySize,expirySize);
    
    idx =1;
    
    for j=1:expirySize-1
        for k=j+1:expirySize
            modelCorr(j,k) = eqCOMDupireSpotGF.computeFuturesCorr(0.0,targetTenors(j),targetTenors(k));
            out(idx) = 1.0*(modelCorr(j,k)-targetCorr(j,k));
            idx = idx + 1;
        end
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsFuturesCorrLevGlobalConstraint(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    targetTenors = params.params.targetTenors;
    targetCorr = params.params.targetCorr;
    % assigning model parameters
    eqCOMDupireSpotGF.alpha = tvar(1);
    eqCOMDupireSpotGF.beta  = tvar(2);
    eqCOMDupireSpotGF.rho   = tvar(3);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(targetTenors);
    
%     numOfEquation = expirySize*(expirySize-1)/2;
    
   % constraint on beta and rho
    numOfEquation = expirySize*(expirySize-1)/2 + 1 +1;
    
    out= zeros(numOfEquation,1);
    
    modelCorr = zeros(expirySize,expirySize);
    
    idx =1;
    
%     smooth_alpha = 1.0;
%     smooth_beta = 10.0;
%     smooth_rho = 10.0;
    
    smooth_alpha = params.params.smooth_alpha;
    smooth_beta = params.params.smooth_beta;
    smooth_rho = params.params.smooth_rho;
    
    alpha_i = params.params.alpha_i;
    beta_i = params.params.beta_i;
    rho_i = params.params.rho_i;
    
    out(idx) = smooth_beta*( eqCOMDupireSpotGF.beta - beta_i);
    idx = idx + 1;
    
    out(idx) = smooth_rho*( eqCOMDupireSpotGF.rho - rho_i);
    idx = idx + 1;
    
    for j=1:expirySize-1
        for k=j+1:expirySize
            modelCorr(j,k) = eqCOMDupireSpotGF.computeFuturesCorr(0.0,targetTenors(j),targetTenors(k));
            out(idx) = 1.0*(modelCorr(j,k)-targetCorr(j,k))/expirySize;
            idx = idx + 1;
        end
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsFuturesCorrLevGlobal(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    targetTenors = params.params.targetTenors;
    targetCorr = params.params.targetCorr;
    % assigning model parameters
    eqCOMDupireSpotGF.alpha = tvar(1);
    eqCOMDupireSpotGF.beta  = tvar(2);
    eqCOMDupireSpotGF.rho   = tvar(3);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(targetTenors);
    numOfEquation = expirySize*(expirySize-1)/2;
    out= zeros(numOfEquation,1);
    
    modelCorr = zeros(expirySize,expirySize);
    
    idx =1;
    
    for j=1:expirySize-1
        for k=j+1:expirySize
            modelCorr(j,k) = eqCOMDupireSpotGF.computeFuturesCorr(0.0,targetTenors(j),targetTenors(k));
            out(idx) = 1.0*(modelCorr(j,k)-targetCorr(j,k));
            idx = idx + 1;
        end
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobalSigmaFixed(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.alpha = tvar(1);
    eqCOMDupireSpotGF.beta  = tvar(2);
    eqCOMDupireSpotGF.rho   = tvar(3);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

    % 1 is constraint on rho
    % 1 is constraint on beta
   
    numOfEquation = expirySize + 1 + 1 + 1;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
    
    % constraint on rho, rho < 0
    smooth_alpha = 10.0;
    out(idx) = smooth_alpha*max((exp(1.0*eqCOMDupireSpotGF.alpha-2.0)-1),0.0);
    idx = idx + 1;
    
    % constraint on rho, rho < 0
    smooth_rho = 10.0;
    out(idx) = smooth_rho*max((exp(1.0*eqCOMDupireSpotGF.rho)-1),0.0);
    idx = idx + 1;
    
    % constraint on beta, beta > 0
%     smooth_beta = 10.0;
    smooth_beta = 10.0;
    
    out(idx) = smooth_beta*max((exp(-1.0*eqCOMDupireSpotGF.beta)-1.0),0.0);
    idx = idx + 1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeSkewFlatModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobalSigmaAlphaFixedConstraint(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.beta  = tvar(1);
    eqCOMDupireSpotGF.rho   = tvar(2);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

   
    numOfEquation = expirySize;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeSkewFlatModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobalSigmaAlphaFixed(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.beta  = tvar(1);
    eqCOMDupireSpotGF.rho   = tvar(2);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

    % 1 is constraint on rho
    % 1 is constraint on beta
   
    numOfEquation = expirySize + 1 + 1;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
    % constraint on rho, rho < 0
    smooth_rho = 10.0;
    out(idx) = smooth_rho*max((exp(1.0*eqCOMDupireSpotGF.rho)-1),0.0);
    idx = idx + 1;
    
    % constraint on beta, beta > 0
    smooth_beta = 10.0;
    out(idx) = smooth_beta*max((exp(-1.0*eqCOMDupireSpotGF.beta)-1),0.0);
    idx = idx + 1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeSkewFlatModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobalLongShort(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.sigma_Short = tvar(1);
    eqCOMDupireSpotGF.sigma_Long = tvar(1 + 1);
    eqCOMDupireSpotGF.beta  = tvar(2 + 1);
    eqCOMDupireSpotGF.rho   = tvar(3 + 1);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

    % 1 is constraint on alpha
    % 2 is constraint on rho
   
    numOfEquation = expirySize + 1 + 1;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
%     % constraint on alpha < 2
%     smooth_alpha = 10.0;
%     out(idx) = smooth_alpha*max((exp(1.0*(eqCOMDupireSpotGF.alpha-2))-1),0.0);
%     idx = idx + 1;
    
    % constraint on rho, rho < 0
    smooth_rho = 10.0;
    out(idx) = smooth_rho*(max((exp(1.0*eqCOMDupireSpotGF.rho-1.0)-1),0.0) + max((exp(-1.0*(eqCOMDupireSpotGF.rho+1))-1),0.0));
    idx = idx + 1;
    
    % constraint on beta, beta > 0
    smooth_beta = 10.0;
    out(idx) = smooth_beta*max((exp(-1.0*eqCOMDupireSpotGF.beta)-1),0.0);
    idx = idx + 1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeLongShortVolModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobalAlphaFixed(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.sigma_flat = tvar(1);
%     eqCOMDupireSpotGF.alpha = tvar(1 + 1);
    eqCOMDupireSpotGF.beta  = tvar(1 + 1);
    eqCOMDupireSpotGF.rho   = tvar(2 + 1);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

    % 1 is constraint on alpha
    % 2 is constraint on rho
   
    numOfEquation = expirySize + 1 + 1;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
    % constraint on rho, rho < 0
    smooth_rho = 10.0;
    out(idx) = smooth_rho*max((exp(1.0*eqCOMDupireSpotGF.rho)-1),0.0);
    idx = idx + 1;
    
    % constraint on beta, beta > 0
    smooth_beta = 10.0;
    out(idx) = smooth_beta*max((exp(-1.0*eqCOMDupireSpotGF.beta)-1),0.0);
    idx = idx + 1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeSkewFlatModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  out  = TargetFunctionsATMVolStripLevGlobal(tvar,eqCOMDupireSpotGF,params)
%     K_swaption = optParams.params.diagSwaption.strike;
%     black_swaption = optParams.params.diagSwaption.blackPrice;
%     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
%     dt= 1.0/(optParams.params.freq);

    expiry = params.params.expiry;
    futuresExpiry = params.params.futuresExpiry;
    marketVol = params.params.atmImpVolStrip;
    
    % assigning model parameters
    
    eqCOMDupireSpotGF.sigma_flat = tvar(1);
    eqCOMDupireSpotGF.alpha = tvar(1 + 1);
    eqCOMDupireSpotGF.beta  = tvar(2 + 1);
    eqCOMDupireSpotGF.rho   = tvar(3 + 1);
    
    % cost function for levenberg marquardt algorithm
    expirySize = length(expiry);
    
%     numOfEquation = expirySize;

    % 1 is constraint on alpha
    % 2 is constraint on rho
   
    numOfEquation = expirySize + 1 + 1 + 1;
    
    out= zeros(numOfEquation,1);
    
    modelVol = zeros(expirySize,1);
    
    idx =1;
    
    % constraint on alpha < 2
    smooth_alpha = 10.0;
    out(idx) = smooth_alpha*max((exp(1.0*(eqCOMDupireSpotGF.alpha-2))-1),0.0);
    idx = idx + 1;
    
    % constraint on rho, rho < 0
    smooth_rho = 10.0;
    out(idx) = smooth_rho*max((exp(1.0*eqCOMDupireSpotGF.rho)-1),0.0);
    idx = idx + 1;
    
    % constraint on beta, beta > 0
    smooth_beta = 10.0;
    out(idx) = smooth_beta*max((exp(-1.0*eqCOMDupireSpotGF.beta)-1),0.0);
    idx = idx + 1;
    
    for j=1:expirySize
        modelVol(j) = eqCOMDupireSpotGF.computeSkewFlatModelVol(expiry(j),futuresExpiry(j));
        out(idx) = 1.0*(modelVol(j)-marketVol(j));
        idx = idx + 1;
        
    end
    
    
end





In [ ]:
function  outVal  = TargetFunctionRoughHestonRGlobalSigurdroemerVal(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqHeston.kappa = tvar(1);
    eqHeston.H = tvar(1);
    eqHeston.rho = tvar(2);
    eqHeston.sigma = tvar(3);
    eqHeston.v0 = tvar(4);
    psiSize = length(eqHeston.psi);
    eqHeston.psi = tvar(5:5 + psiSize -1,1);
    
    addParams = struct;
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i);
           ti  = (0:n) / n * expiry(i)/365.0;
           
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
           
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i);
           ti  = (0:n) / n * expiry(i)/365.0;
           
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
%                     modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
   
   outVal = 0.0;
   idx2 =1;
   for i=1:optParams.params.expirySize
       for j=1:optParams.params.strikeSize
            if optParams.params.calibrationYN(i,j) == 1
                outVal = outVal + out(idx2)*out(idx2);
                idx2 = idx2 + 1;
            end
       end
   end
end





In [ ]:
function  out  = TargetFunctionRoughHestonRGlobalSigurdroemer(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqHeston.H = tvar(1);
%     eqHeston.rho = tvar(2);
%     eqHeston.sigma = tvar(3);
     
    eqHeston.rho = tvar(1);
    eqHeston.sigma = tvar(2);
    eqHeston.H = tvar(3);
    
    eqHeston.v0 = tvar(4);
    psiSize = length(eqHeston.psi);
    eqHeston.psi = tvar(5:5 + psiSize -1,1);
    
    addParams = struct;
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i)*eqHeston.psiDailyTimeStep;
           ti  = (0:n) / n * expiry(i)/365.0;
           
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
           
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i)*eqHeston.psiDailyTimeStep;
           ti  = (0:n) / n * expiry(i)/365.0;
           
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionRoughHestonGlobalSigurdroemerKappa0(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqHeston.kappa = tvar(1);
%     eqHeston.theta = tvar(0);
    eqHeston.rho = tvar(1);
    eqHeston.sigma = tvar(2);
    eqHeston.H = tvar(3);

    eqHeston.v0 = tvar(4);
    
    addParams = struct;
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i);
           ti  = (0:n) / n * expiry(i)/365.0;
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.XicurveKappa0(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
           
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
%TargetFunctionRoughHestonGlobalSigurdroemerHFixed.m

function  out  = TargetFunctionRoughHestonGlobalSigurdroemer(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqHeston.kappa = tvar(1);
    eqHeston.theta = tvar(2);
    eqHeston.rho = tvar(3);
    eqHeston.sigma = tvar(4);
    eqHeston.v0 = tvar(5);
%     eqHeston.H = tvar(5);

    addParams = struct;
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
           n = expiry(i);
           ti  = (0:(n-1)) / n * expiry(i)/365.0;
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
           
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionRoughHestonGlobalSigurdroemer(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqHeston.kappa = tvar(1);
    eqHeston.theta = tvar(1);
    eqHeston.rho = tvar(2);
    eqHeston.sigma = tvar(3);
    eqHeston.v0 = tvar(4);
    eqHeston.H = tvar(5);

    addParams = struct;
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
%            n = expiry(i);
%            n = expiry(i);
           n = 500;
           ti  = (0:n) / n * expiry(i)/365.0;
           dt  = ti(end) - ti(end-1);
           % initial forward variance curve
           xi = eqHeston.Xicurve(ti);
           addParams.n = n;
           addParams.ti = ti;
           addParams.dt = dt;
           addParams.xi = xi;
           
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i),[],addParams);
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionRoughHestonGlobalHFixed(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
%     eqHeston.H = tvar(1);
    eqHeston.rho = tvar(1);
    eqHeston.sigma = tvar(2);
    

    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionRoughHestonGlobal(tvar,eqHeston,optParams)
    
    model_otmPrice = 0.0;
    market_otmPrice = 0.0;

    % assigning model parameters
    eqHeston.H = tvar(1);
    eqHeston.rho = tvar(2);
    eqHeston.sigma = tvar(3);
    

    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqHeston.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_vol = optParams.params.blackVol(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    model_vol = eqHeston.ImpliedBlackVol(model_otmPrice,fwd,marketStrike,expiry(i)/365.0);
                    out(idx) = model_vol - market_vol;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqHeston.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqHeston.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function [ output] = TargetFunctionRebonatoCorrFit3( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    rho_decay = tvar(2);
    alpha = tvar(3);
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            T_i = params.expiry(i);
            T_j = params.expiry(j);
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-rho_decay*exp(-alpha*min(T_i,T_j))*abs(T_i-T_j));
        end
    end
    
    error = 0.0;
    N=0;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            N = N+1;
            error = error +(dummy(i,j)-targetCorr(i,j))*(dummy(i,j)-targetCorr(i,j));
        end
    end
    error =  error/N;
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = error;
end



In [ ]:
function [ output] = TargetFunctionRebonatoCorrFit( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    rho_decay = tvar(2);
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-rho_decay*abs(params.expiry(i)-params.expiry(j)));
        end
    end
    
    error = 0.0;
    N=0;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            N = N+1;
            error = error +(dummy(i,j)-targetCorr(i,j))*(dummy(i,j)-targetCorr(i,j));
        end
    end
    error =  error/N;
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = error;
end



In [ ]:
function [ output] = TargetFunctionRebonatoCorrFit( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    rho_decay = tvar(2);
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-rho_decay*abs(params.expiry(i)-params.expiry(j)));
        end
    end
    
    rebonatoRotM = LoadingM(dummy,2);
    rebonatoCorrM2 = rebonatoRotM*rebonatoRotM';
    
    error = 0.0;
    N=0;
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            N = N+1;
            error = error +(rebonatoCorrM2(i,j)-targetCorr(i,j))*(rebonatoCorrM2(i,j)-targetCorr(i,j));
        end
    end
    error =  error/N;
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = error;
end



In [ ]:
function [ output] = TargetFunctionRebonato3ParamsCorrFitLevmar( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    
    beta_0 = tvar(2);
    beta_1 = tvar(3);
    
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            tau1 = params.expiry(i);
            tau2 = params.expiry(j);
            beta = beta_0*exp(-beta_1*min(tau1,tau2));
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-(beta)*abs(tau1-tau2));
        end
    end
    
    sizeM = (params.sizeN-1)*params.sizeN/2;
    
    out= zeros(sizeM,1);
    
%     out= zeros(6,1);
    idx =1;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            out(idx) = (dummy(i,j)-targetCorr(i,j));
            idx = idx + 1;
        end
    end
    
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = out;
end



In [ ]:
function [ output] = TargetFunctionRebonato3ParamsCorrFitLevmar( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    
    beta_0 = tvar(2);
    beta_1 = tvar(3);
    
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            tau1 = params.expiry(i);
            tau2 = params.expiry(j);
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-(beta_0 + beta_1*max(tau1,tau2))*abs(tau1-tau2));
        end
    end
    
    out= zeros(6,1);
    idx =1;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            out(idx) = (dummy(i,j)-targetCorr(i,j));
            idx = idx + 1;
        end
    end
    
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = out;
end



In [ ]:
function [ output] = TargetFunctionRebonato3ParamsCorrFit( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    
    beta_0 = tvar(2);
    beta_1 = tvar(3);
    
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            tau1 = params.expiry(i);
            tau2 = params.expiry(j);
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-(beta_0 + beta_1*max(tau1,tau2))*abs(tau1-tau2));
        end
    end
    
    error = 0.0;
    N=0;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            N = N+1;
            error = error +(dummy(i,j)-targetCorr(i,j))*(dummy(i,j)-targetCorr(i,j));
        end
    end
    error =  error/N;
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = error;
end



In [ ]:
function [ output] = TargetFunctionRebonato2ParamsCorrFitLevmar( tvar,params)
    expiry  = params.expiry;
    targetCorr = params.targetCorr;
    dummy = zeros(length(expiry),length(expiry));
    rho_inf = tvar(1);
    
    beta_0 = tvar(2);
%     beta_1 = tvar(3);
    
    for i=1:length(params.expiry)
        for j=1:length(params.expiry)
            tau1 = params.expiry(i);
            tau2 = params.expiry(j);
            dummy(i,j) = rho_inf + (1-rho_inf)*exp(-(beta_0)*abs(tau1-tau2));
        end
    end
    
    sizeM = (params.sizeN-1)*params.sizeN/2;
    
    out= zeros(sizeM,1);
    idx =1;
    for i=1:length(params.expiry)-1
        for j=i+1:length(params.expiry)
            out(idx) = (dummy(i,j)-targetCorr(i,j));
            idx = idx + 1;
        end
    end
    
    %UNTITLED2 Summary of this function goes here
    %   Detailed explanation goes here

    output = out;
end



In [ ]:
function  out  = TargetFunctionNormalSABRReducedNormal3(tvar,eqCOMDupireSpotGF,optParams)
    
    % assigning model parameters
   idxNow = optParams.idxNow;
   alpha = tvar(1);
   beta = 0.0;
   rho = tvar(2);
   nu = tvar(3);
    
    % cost function for levenberg marquardt algorithm
   out= zeros(optParams.numOfEquation,1);
   idx =1;
    
   expiry = optParams.params.expiry;
   strike = optParams.params.strike;
%     spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   forward = 0.0;
   optParams.params.calibrationErrorType = 'impVolError';
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       
        for j=1:optParams.params.strikeSize
            market_vol = optParams.params.blackVol(idxNow,j);
            marketStrike = optParams.params.fwdMoneyness(idxNow,j);
            model_vol = SABRNormalVol(0.0,marketStrike,expiry(idxNow)/365.0,alpha,rho,nu);

            out(idx) = model_vol - market_vol;
            idx = idx + 1;
        end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionNormalSABRGlobal3(tvar,eqCOMDupireSpotGF,optParams)
    
    % assigning model parameters
    idxNow = optParams.idxNow;
    alpha = tvar(1);
    beta = 0.0;
    rho = tvar(2);
    nu = tvar(3);
    
    % cost function for levenberg marquardt algorithm
    out= zeros(optParams.numOfEquation,1);
    idx =1;
    
    expiry = optParams.params.expiry;
    strike = optParams.params.strike;
    spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       
        for j=1:optParams.params.strikeSize
            market_vol = optParams.params.blackVol(idxNow,j);
            marketStrike = optParams.params.marketStrike(idxNow,j);
            model_vol = ImpVol(1.0,marketStrike,expiry(i)/365.0);
%                     
%                     if marketStrike <= 1.0
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,-1);
%                     else
%                         model_otmPrice = blackPrice(1.0,marketStrike,model_vol,expiry(i)/365.0,1);
%                     end
%                     
            out(idx) = model_vol - market_vol;
            idx = idx + 1;
        end
     
   elseif strcmp(optParams.params.calibrationErrorType,'priceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice - market_otmPrice;
                    idx = idx + 1;
                end
            end
       end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceErrorSS')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = (model_otmPrice/market_otmPrice - 1.0)*sqrt(expiry(i)/365.0);
                    idx = idx + 1;
                end
            end
        end
   elseif strcmp(optParams.params.calibrationErrorType,'relativePriceError')
       for i=1:optParams.params.expirySize
           fwd = eqCOMDupireSpotGF.Fwd(spot,expiry(i));
            for j=1:optParams.params.strikeSize
                if optParams.params.calibrationYN(i,j) == 1
                    market_otmPrice = optParams.params.blackOTMPrice(i,j);
                    marketStrike = optParams.params.marketStrikes(i,j);
                    model_otmPrice = eqCOMDupireSpotGF.modelOTMPrice(spot,marketStrike,expiry(i));
                    out(idx) = model_otmPrice/market_otmPrice - 1.0;
                    idx = idx + 1;
                end
            end
       end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
function  out  = TargetFunctionLNSABRReducedLN3(tvar,eqCOMDupireSpotGF,optParams)
    
    % assigning model parameters
   idxNow = optParams.idxNow;
   alpha = tvar(1);
   beta = 1.0;
   rho = tvar(2);
   nu = tvar(3);
    
    % cost function for levenberg marquardt algorithm
   out= zeros(optParams.numOfEquation,1);
   idx =1;
    
   expiry = optParams.params.expiry;
   strike = optParams.params.strike;
%     spot =  eqCOMDupireSpotGF.spot.params('rawData');
    
   forward = 0.0;
   optParams.params.calibrationErrorType = 'impVolError';
   if  strcmp(optParams.params.calibrationErrorType,'impVolError')
       
        for j=1:optParams.params.strikeSize
            market_vol = optParams.params.blackVol(idxNow,j);
            marketStrike = optParams.params.fwdMoneyness(idxNow,j);
%             model_vol = SABRNormalVol(0.0,marketStrike,expiry(idxNow)/365.0,alpha,rho,nu);
            model_vol = SABRVol(1.0,marketStrike,expiry(idxNow)/365.0,alpha,beta,rho,nu);

            out(idx) = model_vol - market_vol;
            idx = idx + 1;
        end
   else
       error('unknown calibrationErrorType!')
   end
end





In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
